In [ ]:
import os
from werkzeug.wsgi import SharedDataMiddleware
from flask import Flask, render_template, request, jsonify
from flask_login import LoginManager, login_user, logout_user, login_required, UserMixin
import requests
import bcrypt

app = Flask(__name__)

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'
app.config['SECRET_KEY'] = "lkkajdghdadkglajkgah"

@login_manager.user_loader
def load_user(user_id):
    return User(user_id)

class User(UserMixin):
  def __init__(self,id):
    self.id = id

@app.route("/")
@login_required
def home():
    return render_template('home.html')

@app.route("/list_users")
@login_required
def table():
    headers = {
        'Authorization': 'Bearer your-key',
    }

    r = requests.get('https://api.airtable.com/v0/appM38HXlEVhxmnqx/flaskdemo?sortField=_createdTime&sortDirection=desc', headers=headers)
    dict = r.json()
    dataset = []
    for i in dict['records']:
         dict = i['fields']
         dataset.append(dict)
    return render_template('table.html', entries=dataset)

@app.route("/login")
def login():
    message = 'Please login in first.'
    return render_template('login.html', message=message)

@app.route("/user")
@login_required
def user():
    message = 'Please enter user information.'
    return render_template('userform.html', message=message)

@app.route("/adduser",methods=['POST'])
def adduser():
    mydict =  {
    "fname": request.form['fname'],
    "lname": request.form['lname'],
    "student_id": request.form['student_id'],
    "date_of_birth": request.form['date_of_birth']    
    }
    data = {"fields": mydict}
    headers = {'Authorization': 'Bearer your-key', 'Content-Type': 'application/json; charset=utf-8'}
    r = requests.post('https://api.airtable.com/v0/appM38HXlEVhxmnqx/flaskdemo',json=data,headers=headers)
    return render_template('home.html',message=message)
    
@app.route("/process",methods=['POST'])
def process():
    student_id = request.form['student_id']
    password = request.form['password']
    password = password.encode('UTF-8')
    hashed = ''
    user = ''
    headers = {
    'Authorization': 'Bearer your-key',
    }

    # filter = 'IF(AND({UserName}="'+username+'",{Pwd}="'+password+'"), 1, 0)'
    filter = 'IF(({student_id}="'+student_id+'"), 1, 0)'

    params = (
        ('view', 'Grid view'),
        ('filterByFormula', filter),
    )
    
    dataset = []
    r = requests.session()
    r = requests.get('https://api.airtable.com/v0/appM38HXlEVhxmnqx/flaskdemo?sortField=_createdTime&sortDirection=desc', headers=headers, params=params)           
       
    dict = r.json()

    for i in dict['records']:
         dict = i['fields']
         dataset.append(dict)
    for item in dataset:
        user = item.get('student_id')
        if user != None:
            hashed = item.get('Pwd') 
            fname = item.get('fname')
            hashed = hashed.encode('UTF-8')
        else:
            message = 'wrong password!'
            return render_template('mylogin2.html',message=message)

    if  student_id == user and bcrypt.checkpw(password, hashed):
        login_user(User(1))
        message = "Dear " + fname + ", welcome to Bernard's pages. Your login has been granted."
        return render_template('myhome2.html', message=message)
    message = 'wrong password!'
    return render_template('login.html',message=message)

@app.errorhandler(500)
def internal_error(error):
    message = 'wrong password!'
    return render_template('login.html',message=message),500

@app.route('/logout/')
@login_required
def logout():
    logout_user()
    message = 'Thanks for logging out.'
    return render_template('login.html',message=message)
#@app.route('/')
#def hello_world():
#     return 'Hello from Flask!'

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9013, app)

 * Running on http://localhost:9013/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Sep/2019 16:30:49] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [24/Sep/2019 16:30:49] "GET /login?next=%2F HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2019 16:30:49] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2019 16:30:53] "GET /list_users HTTP/1.1" 302 -
127.0.0.1 - - [24/Sep/2019 16:30:53] "GET /login?next=%2Flist_users HTTP/1.1" 200 -
[2019-09-24 16:31:03,056] ERROR in app: Exception on /process [POST]
Traceback (most recent call last):
  File "/Users/yssuen/anaconda2/envs/py36/lib/python3.6/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/yssuen/anaconda2/envs/py36/lib/python3.6/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/yssuen/anaconda2/envs/py36/lib/python3.6/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb